In [1]:
import pandas as pd

In [20]:
software_url = 'https://docs.google.com/spreadsheets/d/1Sjp9PG75Ap-5YBvOWZ-cCUGkNhN41LZlz3OL-gJ-tKU/export?format=csv&id=1Sjp9PG75Ap-5YBvOWZ-cCUGkNhN41LZlz3OL-gJ-tKU&gid=1702131855'

In [21]:
software = pd.read_csv(software_url)

In [24]:
def build_name_tag_dict(row):
    name = row['Name']
    tags = list(row[['Tag', 'Tag.1', 'Tag.2', 'Tag.3', 'Tag.4', 'Tag.5']])
    
    nt_dict = {'Name': name, 'Tags': tags}
    return nt_dict

In [25]:
software_dict = [build_name_tag_dict(row) for idx, row in software.iterrows()] 

In [ ]:
all_tags_list = software['Tag']